# Caltech Pre-processing
**Purpose: Convert raw Caltech data to YOLO and COCO label formats**</br>
**Final Project**</br>
**Group 1**</br>
**DS6050 Deep Learning**</br>
**Fall 2023**</br>

In [ ]:
%pip install opencv-python pylabel -q

## Download Data

In [ ]:
!wget "https://data.caltech.edu/records/f6rph-90m20/files/data_and_labels.zip?download=1" -O caltechpedestriandataset.zip

## Unpack Data

In [ ]:
!unzip caltechpedestriandataset.zip -d caltechpedestriandataset

## Create YOLO Labels

In [ ]:
!caltech-to-yolo.py

## Move Data (optional)

In [ ]:
!mv datasets/caltechpedestriandataset /scratch/ybt7qf/ds6050-group1-project/datasets --recursive

## Create YOLO Yaml

In [ ]:
%%writefile /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset.yaml
path: /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/
train: /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/images/train
val: /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/images/val
    
nc: 1
    
names: [
    'person'
]

## Convert Training Labels to COCO Format

In [ ]:
!python yolo-to-coco.py \
    --path_to_annotations /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/labels/train/ \
    --path_to_images /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/images/train/ \
    --path_to_yolo_yaml /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset.yaml \
    --name caltechpedestriandataset_train

## Convert Validation Labels to COCO Format

In [ ]:
!python yolo-to-coco.py \
    --path_to_annotations /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/labels/val/ \
    --path_to_images /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/images/val/ \
    --path_to_yolo_yaml /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset.yaml \
    --name caltechpedestriandataset_val

## Move Labels

In [5]:
!mkdir /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/annotations/

In [11]:
!mv /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/labels/train/caltechpedestriandataset_train.json \
    /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/annotations/instances_train2017.json

mv: cannot stat ‘/scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/labels/train/caltechpedestriandataset_train.json’: No such file or directory


In [9]:
!mv /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/labels/val/caltechpedestriandataset_val.json \
    /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/annotations/instances_val2017.json

In [12]:
!ls /scratch/ybt7qf/ds6050-group1-project/datasets/caltechpedestriandataset/annotations

instances_train2017.json  instances_val2017.json
